Sequence Models / NLP
Amaç

Metin tabanlı modelleri (LSTM, GRU, Transformers) uygulayıp sonuçlarını kıyaslamak.

Görev Adımları

Veri Hazırlığı

Kaggle’dan IMDB Sentiment Analysis dataset’ini indir.

Train/test split hazır (varsa tekrar düzenle).

Tokenization ve Padding uygula.

Model 1 – LSTM / GRU

Embedding Layer (örn. 100 boyutlu).

LSTM veya GRU katmanı ekle.

Çıkış: Sigmoid (binary classification).

Model 2 – Transformer Tabanlı

HuggingFace’den BERT veya RoBERTa pre-trained modeli al.

LoRA veya PEFT ile fine-tune et.

Değerlendirme

Accuracy, Precision, Recall, F1 Score hesapla.

BLEU Score (opsiyonel, sequence generation senaryosu için).

Modelleri yan yana kıyasla.

📌 Beklenen Çıktılar: F1 Score tablosu, confusion matrix, model karşılaştırma raporu.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Veri Hazırlığı
- Veri seti text ve label olarak sütunlara ayrıldı 
- "train_test_split" ile %80 train %20 test olarak bölündü.
- stratify = y ile train ve test setlerinde sınıf dağılımı orjinal veri ile aynı tutuldu yani label 1 - 0 sayıları dengeli
- random_state = 42 ile split işlemi tekrarlanabilir yani her çalıştırıldığında aynı train/test bölünmesi elde edilmiş olur

In [2]:
df = pd.read_csv("../data/movie.csv")
X = df["text"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, stratify = y, random_state = 42
)

**HuggingFace + PyTorch**
- Pre-trained tokenizer ile metinler token ID'lerine çevrildi,
- Padding ve truncation ile tüm diziler aynı uzunlukta getirildi,
- Labels tensor formatına dönüştürüldü,
- DataLoader için hazır hale getirildi

In [3]:
from transformers import AutoTokenizer
import torch
import torch.nn as nn

c:\Users\ufuk_delice\Task\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=256, return_tensors="pt")
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=256, return_tensors="pt")

y_train_tensor = torch.tensor(list(y_train.values))
y_test_tensor = torch.tensor(list(y_test.values))

from torch.utils.data import TensorDataset, DataLoader #https://docs.pytorch.org/docs/stable/data.html#module-torch.utils.data

train_dataset = TensorDataset(train_encodings['input_ids'], y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)



### GRU Model 
https://medium.com/@anishnama20/understanding-gated-recurrent-unit-gru-in-deep-learning-2e54923f3e2

#### Model Tanımı

In [5]:
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size=1, num_layers=1, dropout=0.2):
        super(GRUClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(
            input_size = embedding_dim,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True,
            dropout = dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        out, h_n = self.gru(embedded)
        #out[:, -1, :] -> son time step'i al
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

#### Loss ve Optimizer

In [6]:
model = GRUClassifier(vocab_size = len(tokenizer), embedding_dim = 100, hidden_size = 128)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)

#### Dataset ve DataLoader

In [8]:
train_dataset = TensorDataset(train_encodings['input_ids'], y_train_tensor)
test_dataset = TensorDataset(test_encodings['input_ids'], y_test_tensor)

traind_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32)

#### Training Loop

In [9]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6937
Epoch 2, Loss: 0.5678
Epoch 3, Loss: 0.3130


#### Evaluation

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, dataloader):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            preds.extend((outputs.squeeze() > 0.5).int().tolist())
            true_labels.extend(labels.tolist())
    acc = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    cm = confusion_matrix(true_labels, preds)
    return acc, prec, rec, f1, cm


#### Test Dataset ve DataLoader

In [14]:
accuracy, precision, recall, f1, confusion = evaluate(model, test_loader)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)


Accuracy: 0.869625
Precision: 0.8733249051833123
Recall: 0.8643643643643644
F1 Score: 0.868821531882782
Confusion Matrix:
 [[3503  501]
 [ 542 3454]]


### LSTM Model
https://medium.com/@techwithjulles/recurrent-neural-networks-rnns-and-long-short-term-memory-lstm-creating-an-lstm-model-in-13c88b7736e2

#### Model Tanımı

In [16]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size=1, num_layers=1, dropout=0.2):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        embedded = self.embedding(x)
        out, (h_n, c_n) = self.lstm(embedded)
        out = self.fc(out[:, -1, :])  # son time step’i al
        out = self.sigmoid(out)
        return out

#### Loss ve Optimizer

In [17]:
model = LSTMClassifier(vocab_size=len(tokenizer), embedding_dim=100, hidden_size=128)
criterion = nn.BCELoss()  # veya BCEWithLogitsLoss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


#### Dataset ve DataLoader

In [18]:
train_dataset = TensorDataset(train_encodings['input_ids'], y_train_tensor)
test_dataset  = TensorDataset(test_encodings['input_ids'], y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32)

#### Training Loop

In [19]:
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: 0.6930
Epoch 2, Loss: 0.6617
Epoch 3, Loss: 0.5753


#### Evaluation

In [20]:
def evaluate(model, dataloader):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            preds.extend((outputs.squeeze() > 0.5).int().tolist())
            true_labels.extend(labels.tolist())
    return preds, true_labels


#### Test Dataset ve DataLoader

In [21]:
preds, true_labels = evaluate(model, test_loader)

print("Accuracy:", accuracy_score(true_labels, preds))
print("Precision:", precision_score(true_labels, preds))
print("Recall:", recall_score(true_labels, preds))
print("F1 Score:", f1_score(true_labels, preds))
print("Confusion Matrix:\n", confusion_matrix(true_labels, preds))

Accuracy: 0.79
Precision: 0.7698042870456664
Recall: 0.8268268268268268
F1 Score: 0.7972972972972973
Confusion Matrix:
 [[3016  988]
 [ 692 3304]]


## GRU & LSTM Karşılaştırması

| Metric           | GRU                        | LSTM                       |
| ---------------- | -------------------------- | -------------------------- |
| Accuracy         | 0.8696                     | 0.79                       |
| Precision        | 0.8733                     | 0.7698                     |
| Recall           | 0.8644                     | 0.8268                     |
| F1 Score         | 0.8688                     | 0.7973                     |
| Confusion Matrix | [[3503, 501], [542, 3454]] | [[3016, 988], [692, 3304]] |



1. GRU daha yüksek Accuracy ve F1 Score verdi
    - Bu veri seti (IMDB, yorumlar kısa/orta uzunlukta) GRU için daha uygun olabilir.
2. LSTM Recall biraz yüksek ama Precision düşük
    - Yani LSTM pozitifleri daha fazla yakalıyor ama yanlış pozitif tahminleri de artırıyor.
3. Confusion Matrix farkı
    - LSTM’de FP = 988 → GRU’ya göre neredeyse iki kat
    - FN = 692 → LSTM biraz daha fazla kaçırıyor
4. Genel Yorum
    - Bu veri setinde GRU daha dengeli ve başarılı çıktı.
    - LSTM’in avantajı genellikle çok uzun sekanslar ve daha karmaşık bağımlılıklar için ortaya çıkar.


## Transformer

#### HuggingFace Tokenizer
https://medium.com/data-science-collective/how-to-fine-tune-an-llm-on-your-data-with-hugging-face-lora-563e57ca8c1c

In [36]:
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model
from torch.utils.data import Dataset

Pre-trained BERT modeli ve tokenizer

In [34]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LoRA Konfigürasyonu

In [35]:
lora_config = LoraConfig(
    r=8,                  # low-rank boyutu
    lora_alpha=32,        # ölçekleme faktörü
    target_modules=["query", "value"],  # hangi ağırlıklara LoRA uygulanacak
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(model, lora_config)

Dataset & DataLoader

In [37]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    


train_dataset = IMDbDataset(train_encodings, y_train.values)
test_dataset  = IMDbDataset(test_encodings, y_test.values)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=8, num_workers=0)


Training Loop / Trainer Setup

In [38]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)


Training Loop

In [ ]:
num_epochs = 1  # CPU için başlangıçta 1 epoch yeterli
device = torch.device("cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        
        # Batch'i cihaza taşı
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels)
        
        # Backward pass ve optimizer step
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


Evaluation Loop

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, dataloader):
    model.eval()
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            pred_labels = torch.argmax(logits, dim=1)
            
            preds.extend(pred_labels.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    
    acc = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds)
    rec = recall_score(true_labels, preds)
    f1 = f1_score(true_labels, preds)
    cm = confusion_matrix(true_labels, preds)
    return acc, prec, rec, f1, cm

accuracy, precision, recall, f1, confusion = evaluate(model, test_loader)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", confusion)
